# ReLU && Optimizer

## sigmoid의 문제점

#### sigmoid를 activation function으로 사용한다면 어떻게 될까?

- weight, input를 이용한 sigmoid함수로 output을 구한다. 
- 실제 정답과의 차이를 이용해 Loss를 구한다. 
- Loss를 미분하여 gradient를 구한다. 
- back propagation을 이용해 weight를 업데이트 한다. 

#### 무엇이 문제일까? 

- gradient를 구할 때 문제가 된다. 
- sigmoid함수 그래프를 보면 이해가 쉬워진다. 

![sigmoid](../statics/image/sigmoid.jpg)

- x가 0에 가까울수록 gradient가 확연하게 바뀐다. 
- 양 끝의 경우, 소수점 뒤의 수만 작게 변하게 된다. 
- 문제는, back propagation을 이용해 gradient를 전파하고 activation function에서 전파받은 gradient가 무의미해진다(vanishing gradient)
- sigmoid함수를 이용하는 레이어가 중첩될수록 더 큰 영향을 끼치게 된다. 

#### ReLU는 무엇이 다르길래 괜찮은걸까?

`f(x) = max(0, x)`

- ReLU는 0보다 작으면 무조건 0을, 크면 x를 그대로 반환한다. 
- x > 0 일 경우, gradient는 항상 1이다. 
- 음수의 영역에선 gradient가 항상 0이기 떄문에, 음수의 x에 대해선 vanishing gradient가 여전히 발생한다. 
- pytorch에서는 `torch.nn.relu(x)`를 이용한다. 

#### 다양한 활성화 함수

- sigmoid
    - 우리가 아는 그것. 1 / (1 + e^(-x))
- tanh
    - (e^x - e(-x)) / (e^x + e^(-x))
    - sigmoid보다 출력 중심이 0이라 성능을 개선할 수 있다. 
    - vanishing gradient가 여전히 존재한다. 
- relu
    - 현재 가장 상용화되어있으며, 양수인 x에 항상 사용 가능하다. 
    - 입력이 0보다 작으면 dead neuron이 발생할 수 있다. 
- leaky_relu
    - f(x) = max(ax, x) a에 따라 음수의 영역에서도 아주 작게나마 gradient를 가진다. 

## Optimizer

- 손실함수의 값을 줄이기 위해 가중치를 어떻게 업데이트할까?를 결정하는 역할
- torch.optim에 optimizer가 다양하게 존재한다. 

#### 최적화 알고리즘의 종류
- torch.optim.SGD(Stochastic Gradient Descent)
    - 한 번에 하나의 샘플 데이터를 이용한다. 
    - 간단하다
    - 학습 튜닝이 필요하며, 수렴까지 오래걸릴 수 있다. 
- torch.optim.Adagrad
    - 과거 기울기를 누적해 learning rate를 자동 조정한다.  
    - 학습 데이터가 적을 때 효과적이다. 
    - lr이 점점 0에 수렴하여 학습을 멈출 수 있다. 
- torch.optim.Adadelta
    - Adadelta를 개선하여, learning rate를 0이 되지 않도록 RMS기반 조정을 수행한다.( Root Mean Square : 기울기 제곱의 평균값의 루트)
    - learing rate를 튜닝할 필요가 없다. 
    - 구조가 복잡하다.
- torch.optim.RMSprop
    - Adadelta와 유사하다. 최근 기울기에 더 큰 가중치를 둬 학습률을 조정한다. 
    - RNN에 강하다. (RNN이 뭘까?)
    - 모멘텀 조합이 필요할 수 있다. (모멘텀이 뭘까?) 
- torch.optim.Adam
    - 모멘텀 + RMSprop조합으로, 가장 널리 사용된다.
    - 수렴이 빠르고 튜닝이 거의 필요 없다. 
    - 간혹 촤솟값 근처에서 진동형 변동이 일어난다. (와리가리)
- torch.optim.Adamax
    - Adam 변형으로, 무한노름기반 안정화된 버전이다. (무한노름이 뭘까?)  
    - 고차원 공간에 최적화되어있다. 
    - 일반 Adam보다 사용 빈도가 낮다. 
- torch.optim.SparseAdam
    - sparse tensor용 Adam이다. (sparse tensor가 뭘까?)
    - 회소 벡터 최적화에 강하다. 
    - Dense에는 부적합하다. (Dense가 뭘까?)
- torch.optim.ASGD
    - 평균 파라미터를 이용한 SGD의 변형버전
    - 수렴 안정성이 증가한다. 
    - 잘 안쓴다. 
- torch.optim.LBFGS
    - 2차 최적화(준 뉴턴 방식), 배치 전체 기반 (준 뉴턴방식이 뭘까?)
    - 작은 모델에선 빠른 수렴을 할 수 있다. 
    - 메모리 부하가 크고, 미니배치에 부적합하다
- torch.optim.Rprop
    - 기울기 크기 무시하고 방향만을 이용한다(단위 벡터 쓰는건가?)
    - 고정된 step size 기반이다. 
    - SGD보다 느리고, 잘 쓰지 않는다. 

공부하다보니, 또 다른 모르는 단어들이 나왔다. 한번 찾아봐야겠다. 

- RNN
- 모멘텀
- 무한노름
- sparse tensor
- Dense
- 준 뉴턴 방식
- Rprop -> 기울기 크기를 무시하고 방향만을 이용하면 단위벡터화시킨걸까?